In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Colab Notebooks/AI Projects/HealthCare/Brain_MRI

/content/drive/My Drive/Colab Notebooks/AI Projects/HealthCare/Brain_MRI


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import files

In [4]:
brain_df= pd.read_csv('data_mask.csv')

In [5]:
brain_df.head()

,patient_id,image_path,mask_path,mask
0,TCGA_CS_5395_19981004,TCGA_CS_5395_19981004/TCGA_CS_5395_19981004_1.tif,TCGA_CS_5395_19981004/TCGA_CS_5395_19981004_1_...,0
1,TCGA_CS_5395_19981004,TCGA_CS_4944_20010208/TCGA_CS_4944_20010208_1.tif,TCGA_CS_4944_20010208/TCGA_CS_4944_20010208_1_...,0
2,TCGA_CS_5395_19981004,TCGA_CS_4941_19960909/TCGA_CS_4941_19960909_1.tif,TCGA_CS_4941_19960909/TCGA_CS_4941_19960909_1_...,0
3,TCGA_CS_5395_19981004,TCGA_CS_4943_20000902/TCGA_CS_4943_20000902_1.tif,TCGA_CS_4943_20000902/TCGA_CS_4943_20000902_1_...,0
4,TCGA_CS_5395_19981004,TCGA_CS_5396_20010302/TCGA_CS_5396_20010302_1.tif,TCGA_CS_5396_20010302/TCGA_CS_5396_20010302_1_...,0


In [6]:
brain_df= brain_df.drop(columns=['patient_id'])

In [15]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(brain_df, test_size = 0.20)

In [16]:
from keras.preprocessing.image import ImageDataGenerator
datagen= ImageDataGenerator(rescale=1./255., validation_split = 0.20)

In [17]:
brain_df['mask'] = brain_df['mask'].apply(lambda x: str(x))

In [19]:
train_generator = datagen.flow_from_dataframe(
dataframe=train,
directory= './',
x_col='image_path',
y_col='mask',
subset="training",
batch_size=15,
shuffle=True,
class_mode="categorical",
target_size=(256,256))

valid_generator=datagen.flow_from_dataframe(
dataframe=train,
directory= './',
x_col='image_path',
y_col='mask',
subset="validation",
batch_size=16,
shuffle=True,
class_mode="categorical",
target_size=(256,256))

# Create a data generator for test images
test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
dataframe=test,
directory= './',
x_col='image_path',
y_col='mask',
batch_size=16,
shuffle=False,
class_mode='categorical',
target_size=(256,256))





Found 1335 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 1475 invalid image filename(s) in x_col="image_path". These filename(s) will be ignored.
  warnings.warn(


Found 333 validated image filenames belonging to 2 classes.
Found 419 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 1475 invalid image filename(s) in x_col="image_path". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 367 invalid image filename(s) in x_col="image_path". These filename(s) will be ignored.
  warnings.warn(


In [21]:
#Importing resnet and creating resnet with imagenet
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import Input
basemodel = ResNet50(weights = 'imagenet', include_top = False, input_tensor = Input(shape=(256, 256, 3)))

94765736/94765736 [==============================] - 1s 0us/step


In [22]:
basemodel.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 262, 262, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 128, 128, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 128, 128, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [24]:

for layer in basemodel.layers:
  layer.trainable = False

In [26]:
#Adding Classifier Layer to Basemodel
from tensorflow.keras.layers import AveragePooling2D,Dense,Dropout,Flatten
from tensorflow.keras.models import Model
headmodel = basemodel.output
headmodel = AveragePooling2D(pool_size = (4,4))(headmodel)
headmodel = Flatten(name= 'flatten')(headmodel)
headmodel = Dense(256, activation = "relu")(headmodel)
headmodel = Dropout(0.3)(headmodel)#
headmodel = Dense(256, activation = "relu")(headmodel)
headmodel = Dropout(0.3)(headmodel)
headmodel = Dense(2, activation = 'softmax')(headmodel)

model = Model(inputs = basemodel.input, outputs = headmodel)

In [27]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 262, 262, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 128, 128, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 128, 128, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [28]:
#Compiling the mode
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [30]:
from tensorflow.keras.callbacks import EarlyStopping
earlystopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

In [31]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath="classifier-resnet-weights.hdf5", verbose=1, save_best_only=True)


In [32]:
history = model.fit(train_generator, steps_per_epoch= train_generator.n // 16, epochs = 1, validation_data= valid_generator, validation_steps= valid_generator.n // 16, callbacks=[checkpointer, earlystopping])

83/83 [==============================] - ETA: 0s - loss: 0.8420 - accuracy: 0.5847
Epoch 1: val_loss improved from inf to 0.65496, saving model to classifier-resnet-weights.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


83/83 [==============================] - 435s 5s/step - loss: 0.8420 - accuracy: 0.5847 - val_loss: 0.6550 - val_accuracy: 0.6406


In [33]:
model_json = model.to_json()
with open("classifier-resnet-model.json","w") as json_file:
  json_file.write(model_json)

In [36]:
import tensorflow as tf
with open('resnet-50-MRI.json', 'r') as json_file:
    json_savedModel= json_file.read()
# load the model
model = tf.keras.models.model_from_json(json_savedModel)
model.load_weights('weights.hdf5')
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics= ["accuracy"])

In [37]:
#Predict model
test_predict = model.predict(test_generator, steps = test_generator.n // 16, verbose =1)

26/26 [==============================] - 138s 5s/step


In [38]:
predict =[]
for i in test_predict:
  predict.append(str(np.argmax(i)))

predict = np.asarray(predict)


In [39]:
predict

array(['0', '0', '1', '0', '0', '0', '1', '0', '1', '1', '0', '0', '0',
       '1', '1', '1', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0',
       '1', '0', '0', '0', '0', '1', '1', '1', '1', '1', '1', '0', '1',
       '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '0', '0',
       '1', '1', '0', '1', '0', '0', '0', '0', '0', '1', '1', '0', '1',
       '0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '1', '0',
       '0', '0', '1', '1', '1', '0', '1', '0', '1', '1', '0', '1', '0',
       '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '1',
       '1', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0',
       '0', '0', '1', '1', '0', '0', '1', '0', '1', '0', '1', '0', '1',
       '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0',
       '1', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '1',
       '0', '0', '0', '1', '1', '0', '0', '1', '0', '1', '0', '0', '1',
       '0', '1', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0

In [41]:
original=[]
original=np.asarray(test['mask'])[:len(predict)]

In [42]:
original

array(['0', '0', '0', '1', '0', '0', '0', '1', '0', '1', '1', '0', '1',
       '0', '0', '1', '1', '0', '0', '0', '1', '1', '1', '1', '0', '1',
       '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0',
       '1', '0', '1', '1', '0', '0', '0', '0', '1', '1', '1', '1', '1',
       '1', '1', '0', '1', '0', '0', '1', '0', '0', '0', '0', '1', '1',
       '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '1', '1', '0', '0', '0', '1', '1', '0', '0', '0', '1', '0', '1',
       '0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '0', '0', '1',
       '0', '0', '0', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0',
       '1', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '1',
       '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '1',
       '0', '1', '0', '0', '1', '1', '0', '1', '1', '0', '1', '0', '1',
       '1', '1', '1', '1', '0', '1', '1', '1', '0', '0', '0', '0', '1',
       '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '0', '1

In [43]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(original,predict)

In [44]:
cm

array([[169,  96],
       [ 98,  53]])

In [45]:
from sklearn.metrics import accuracy_score
score = accuracy_score(original,predict)

In [46]:
score

0.5336538461538461